In [1]:
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras as K


In [2]:
from keras.datasets import cifar10
from keras import initializers

import copy

In [3]:
# start building the model - import necessary layers
from keras.models import Sequential
from keras.layers import Dropout, Activation, Conv2D, GlobalAveragePooling2D
from keras.optimizers import SGD

In [4]:
use_colab = True
assert use_colab in [True, False]

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [7]:
print ("Training data:")
print ("Number of examples: ", X_train.shape[0])
print ("Number of channels:",X_train.shape[3]) 
print ("Image size:", X_train.shape[1], X_train.shape[2])
print
print ("Test data:")
print ("Number of examples:", X_test.shape[0])
print ("Number of channels:", X_test.shape[3])
print ("Image size:", X_test.shape[1], X_test.shape[2]) 

print(X_train.shape, X_train.dtype)

Training data:
Number of examples:  50000
Number of channels: 3
Image size: 32 32
Test data:
Number of examples: 10000
Number of channels: 3
Image size: 32 32
(50000, 32, 32, 3) uint8


In [8]:
# normalize the inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [9]:
# the save point
if use_colab:
    checkpoint_dir ='/content/drive/My Drive/0003. AICC 자료/001. Study 자료/Project_Code/CIFAR-10/datagen/exp1'
else:
    checkpoint_dir = 'datagen/exp1'

if not os.path.isdir(checkpoint_dir):
        os.makedirs(checkpoint_dir)    

https://www.kaggle.com/aarooxx/object-recognition-90-88-accuracy

kaggle acc : 90.88% Code reference

In [10]:
model = Sequential()

# add model layers - Convolution2D, Activation, Dropout
model.add(Conv2D(96, (3, 3), padding = 'same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(96, (3, 3), padding = 'same', strides = (2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (3, 3), padding = 'same', strides = (2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(192, (3, 3), padding = 'same'))
model.add(Activation('relu'))
model.add(Conv2D(192, (1, 1), padding = 'valid'))
model.add(Activation('relu'))
model.add(Conv2D(10, (1, 1), padding = 'valid'))

# add GlobalAveragePooling2D layer with Softmax activation
model.add(GlobalAveragePooling2D())
model.add(Activation('softmax'))

In [11]:
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics='categorical_accuracy')

In [12]:
import keras.utils

one_hot_y_train = keras.utils.to_categorical(y_train, num_classes=10)

In [13]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)

In [16]:
# Direct Dataload

model.fit(X_train, one_hot_y_train, epochs=40, batch_size=1000, validation_split=0.2
          ,callbacks=[cp_callback])

Epoch 1/40
40/40 [==============================] - ETA: 0s - loss: 0.5294 - categorical_accuracy: 0.8140WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0072s vs `on_test_batch_end` time: 0.0953s). Check your callbacks.

Epoch 00001: val_loss improved from 0.66351 to 0.65352, saving model to /content/drive/My Drive/0003. AICC 자료/001. Study 자료/Project_Code/CIFAR-10/datagen/exp1
40/40 [==============================] - 15s 387ms/step - loss: 0.5294 - categorical_accuracy: 0.8140 - val_loss: 0.6535 - val_categorical_accuracy: 0.7758
Epoch 2/40
40/40 [==============================] - ETA: 0s - loss: 0.5016 - categorical_accuracy: 0.8242
Epoch 00002: val_loss did not improve from 0.65352
40/40 [==============================] - 16s 398ms/step - loss: 0.5016 - categorical_accuracy: 0.8242 - val_loss: 0.6890 - val_categorical_accuracy: 0.7679
Epoch 3/40
40/40 [==============================] - ETA: 0s - loss: 0.4972 - categorical_

In [17]:
# 최적 weight load

model.load_weights(checkpoint_dir)

In [19]:
one_hot_y_test = keras.utils.to_categorical(y_test, num_classes=10)

model.evaluate(X_test, one_hot_y_test, batch_size=1000)

10/10 [==============================] - 1s 108ms/step - loss: 0.6158 - categorical_accuracy: 0.7950


[0.6157543659210205, 0.7950000166893005]

Model Ver.2 : Datagen

In [15]:
# Datagen 모델 

(X_train, y_train), (X_test, y_test) = cifar10.load_data()


In [ ]:

from sklearn.model_selection import train_test_split

# datagenerator 사용을 위해 validation set split
X_train, X_valid, Y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)


valid_datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)



In [15]:
train_datagen.fit(X_train)

valid_datagen.fit(X_valid)

In [ ]:
# model compile
model.compile(loss='categorical_crossentropy', 
                  optimizer='adam',
                  metrics='categorical_accuracy')

In [ ]:
import keras.utils

one_hot_y_train = keras.utils.to_categorical(y_train, num_classes=10)

In [ ]:

# model fit - using datagen flow

model.fit(train_datagen.flow(X_train, y_train, batch_size=1000) ,
          step_per_epoch = 40,  # 40000 / 1000
          epochs=40, 
          validation_data = valid_datagen.flow(X_valid,y_valid, batch_size=1000) ,
          validation_step = 1,
          callbacks=[cp_callback])
